In [1]:
import pandas as pd
import numpy as np
import os
from utils import base

In [2]:
path = "./tctodd/"
#path = "../../Desktop/MML Project/tctodd/"
dirs = os.listdir(path=path)
weeks = sorted([i for i in dirs if i != ".DS_Store"])
filenames = sorted(os.listdir(path+weeks[1]))

data = []
labels = dict()
label_cnt = 0

for w in weeks:
    temp_path = path+w+"/"
    filenames = sorted(os.listdir(temp_path))
    for fn in filenames:
        label = fn.split('.')[0][:-2]
        
        if label not in labels:
            labels[label] = label_cnt
            label_cnt += 1
            
        data.append({'label':labels[label], 'time_series':pd.read_csv(temp_path+fn, header=None, sep='\t',).values})
        

In [51]:
from sklearn.model_selection import train_test_split
from tslearn.utils import to_time_series_dataset
df = pd.DataFrame(data, columns=['label', 'time_series'])
seed = 0
X = df['time_series']
y = df['label']

X = to_time_series_dataset(X)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=0.222222222222222, random_state=seed)

In [55]:
from tslearn.svm import TimeSeriesSVC
import sklearn as sk
from sklearn.metrics import accuracy_score
import itertools
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from tslearn.utils import to_time_series_dataset
df = pd.DataFrame(data, columns=['label', 'time_series'])
seed = 0
X = df['time_series']
y = df['label']

X = to_time_series_dataset(X)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=0.222222222222222, random_state=seed)

# Scaling

# Compute mean and std without taking care of nans
X_train_no_nan = []
for i in range(X_train.shape[0]):
    sample = X_train[i]
    mask = ~np.isnan(sample)
    sample_no_nan = sample[mask].reshape(-1,22)
    X_train_no_nan.append(sample_no_nan)
    

X_train_no_nan = np.vstack(X_train_no_nan)
mean = np.mean(X_train_no_nan, axis=0)
std = np.std(X_train_no_nan, axis=0)

# Scale everything (train)
for i in range(X_train.shape[0]):
    sample = X_train[i]
    # select entries != nan
    mask = ~np.isnan(sample)
    sample_no_nan = sample[mask].reshape(-1,22)
    scaled = (sample_no_nan - mean) / std
    X_train[i].ravel()[:len(scaled.ravel())] = scaled.ravel()

# Scale everything (test)
for i in range(X_test.shape[0]):
    sample = X_test[i]
    mask = ~np.isnan(sample)
    sample_no_nan = sample[mask].reshape(-1,22)
    scaled = (sample_no_nan - mean) / std
    X_test[i].ravel()[:len(scaled.ravel())] = scaled.ravel()

clf = TimeSeriesSVC()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
res = accuracy_score(y_test, predictions)

print(res)

0.8701754385964913


In [5]:
'''mean_ary = np.zeros(shape=(22,))
weights_ary = np.zeros(shape=(22))
std_ary = np.zeros(shape=(22,))'''

# Compute mean and std without taking care of nans
X_train_no_nan = []
for i in range(X_train.shape[0]):
    sample = X_train[i]
    mask = ~np.isnan(sample)
    sample_no_nan = sample[mask].reshape(-1,22)
    X_train_no_nan.append(sample_no_nan)
    

X_train_no_nan = np.vstack(X_train_no_nan)
mean = np.mean(X_train_no_nan, axis=0)
std = np.std(X_train_no_nan, axis=0)

#print(mean, std)


# Scale everything
for i in range(X_train.shape[0]):
    sample = X_train[i]
    #print(sample.shape)
    mask = ~np.isnan(sample)
    sample_no_nan = sample[mask].reshape(-1,22)
    scaled = (sample_no_nan - mean) / std
    #print(scaled)
    #print(scaled.shape)
    X_train[i].ravel()[:len(scaled.ravel())] = scaled.ravel()
    
X_train[0]

array([[-0.61047705, -0.57930302, -0.07565125, ...,  0.01096043,
        -0.4532766 , -0.73419219],
       [-0.57661299, -0.60673875, -0.09003222, ...,  0.54181317,
        -0.4532766 , -0.65938806],
       [-0.57252357, -0.86328223, -0.0879778 , ...,  1.04606729,
        -0.4532766 , -0.33016164],
       ...,
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan]])

In [54]:
np.sum(X_train_no_nan,axis=0)

array([-0.05610578, -0.11313968, -0.13779364,  0.5992977 ,  0.30666848,
        0.46251229,  0.08999458,  0.05210866,  0.02986489,  0.05501536,
        0.07046579,  0.05096768, -0.0443975 , -0.1206937 ,  0.43741315,
        0.39030944,  0.5483283 ,  0.27882825,  0.05666373,  0.22534062,
        0.13895616,  0.26285182])

# previous results

In [ ]:
from sklearn.model_selection import train_test_split
from tslearn.utils import to_time_series_dataset
df = pd.DataFrame(data, columns=['label', 'time_series'])
seed = 0
X = df['time_series']
y = df['label']

X = to_time_series_dataset(X)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=0.222222222222222, random_state=seed)

In [4]:
from tslearn.svm import TimeSeriesSVC
import sklearn as sk
from sklearn.metrics import accuracy_score
import itertools
from tqdm import tqdm

n_folds = 5
skf = sk.model_selection.StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
params = [[1e-4, 1e-2, 1], [1e-1, 1, 10]]
params_comb = list(itertools.product(*params))

acc_scores = np.zeros(len(params_comb))
for train_index, val_index in skf.split(X_train, y_train):
    X_train_cv = X_train[train_index]
    y_train_cv = y_train.iloc[train_index]
    X_val_cv = X_train[val_index]
    y_val_cv = y_train.iloc[val_index]
    
    mean_ary, std_ary = base.compute_mean_and_std(X_train_cv)

    # Scale X_train_cv 
    X_train_cv = base.apply_z_standardization(X_train_cv, mean_ary, std_ary)
    # Scale X_test_cv
    X_val_cv = base.apply_z_standardization(X_val_cv, mean_ary, std_ary)
    
    for i, params in enumerate(tqdm(params_comb, desc='doing parameters search...')):
        clf = TimeSeriesSVC(C=params[0], gamma=params[1], kernel="gak", max_iter=1000)
        clf.fit(X_train_cv, y_train_cv)
        predictions = clf.predict(X_val_cv)
        res = accuracy_score(y_val_cv, predictions)
        acc_scores[i] += res
acc_scores = acc_scores / n_folds
best_idx = np.argmax(acc_scores)    
print(f'Found best combination! {params_comb[best_idx]} w. accuracy of {acc_scores[best_idx]}.')
best_comb = params_comb[best_idx]

doing parameters search...: 100%|██████████| 9/9 [4:07:08<00:00, 1647.62s/it]  

Found best combination! (1, 10) w. accuracy of 0.6962406015037594.


In [5]:
mean_ary, std_ary = base.compute_mean_and_std(X_train)

# Scale X_train_cv 
X_train = base.apply_z_standardization(X_train, mean_ary, std_ary)
# Scale X_test_cv
X_test = base.apply_z_standardization(X_test, mean_ary, std_ary)

clf = TimeSeriesSVC(C=best_comb[0], gamma=best_comb[1], kernel="gak")
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
res = accuracy_score(y_test, predictions)
print(f'Reached an accuracy of {res}.')

Reached an accuracy of 0.6859649122807018.
